In [1]:

from tqdm import tqdm
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from pathlib import Path
import spacy

In [ ]:
data_list = []

# TODO read it in as spacy docs
# , then remove digits
# , maybe remove verbs
# , then run the topic modeling
# , remove names
# , romanization cohesiveness


# TODO feedback:
# chunk the text (maybe set a chunk min size for shorter texts)
# , spacy filtering
# , lemmatization
# , be more aggressive w/ max_df arg
# , set a min number of times a word has to be used


nlp = spacy.load("en_core_web_sm")


for path in Path("sloppy_ocr_manual_text").glob("*.txt"):
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read()
        text = text.replace("\n", " ") #just in case some nonsense occurred in the ocr
    
    data_list.append((path.name, text))

In [3]:
df = pd.DataFrame(data_list, columns=['file_name', 'text'])

In [4]:
vectorizer = TfidfVectorizer(min_df=0.1, max_df=0.8, stop_words="english")
vectorized_data = vectorizer.fit_transform(df["text"])

In [26]:
nmf = NMF(n_components=20, random_state=42)

In [27]:
doc_topic_distribution = nmf.fit_transform(vectorized_data)

In [28]:
topic_words_df = pd.DataFrame(nmf.components_ , columns=vectorizer.get_feature_names_out())

In [29]:
topics = []
for topic, topic_row in topic_words_df.iterrows():
    top_10_words = topic_row.sort_values(ascending=False).head(10).index.tolist()
    topic_label = ", ".join(top_10_words)
    print(f"Topic {topic}: {topic_label}")
    topics.append(topic_label)


Topic 0: kim, yi, western, japanese, school, chosŏn, tokyo, japan, oil, 15
Topic 1: japanese, imperial, modernism, difference, universality, spectacle, dissertation, era, transnational, promise
Topic 2: dansaekhwa, movement, kukje, gallery, exhibition, traditional, making, using, yoon, studio
Topic 3: contemporary, kee, war, ideology, argues, suspended, historical, north, 1950, joan
Topic 4: film, cinema, films, hong, soo, lee, sang, dong, box, poetry
Topic 5: horlyck, chapters, book, textbook, contemporary, developments, pp, available, interpretations, situates
Topic 6: self, portraiture, portrait, na, ko, western, artist, autonomous, oil, academic
Topic 7: research, laboratory, table, woman, yi, nihonga, late, glass, ae, experiment
Topic 8: experimental, kim, class, work, middle, state, university, society, does, mean
Topic 9: ink, jung, abstraction, suh, euro, work, like, nature, contemporary, bienal
Topic 10: kisaeng, female, women, beauty, femininity, figure, depictions, models, t

In [30]:
doc_topic_df = pd.DataFrame(doc_topic_distribution, columns=topics)

for doc_index, doc_row in doc_topic_df.iterrows():
    top_3_topics = doc_row.sort_values(ascending=False).head(3)

    doc_info = df.loc[doc_index, ['file_name']]
    print(f"\n Top 3 topics for file {doc_info['file_name']}: ")
    for topic, weight in top_3_topics.items():
        print(f'{topic} {weight}')


 Top 3 topics for file art_of_dansaekhwa_press_release.txt: 
dansaekhwa, movement, kukje, gallery, exhibition, traditional, making, using, yoon, studio 0.777107843349224
contemporary, kim, ink, seoul, like, ŭi, yi, world, 16, 1960s 8.429595250952039e-10
landscape, sketch, picturesque, japanese, tour, imperial, east, aesthetic, mountains, asian 9.748796007085404e-11

 Top 3 topics for file Book_Reviews—Korea_Korean_Art.txt: 
horlyck, chapters, book, textbook, contemporary, developments, pp, available, interpretations, situates 0.5830504715569493
contemporary, kim, ink, seoul, like, ŭi, yi, world, 16, 1960s 7.376469655575642e-12
abstraction, war, international, experimental, global, internationalization, 1970s, late, monochrome, participation 3.423984217787056e-12

 Top 3 topics for file companion_to_korean_art_chap15.txt: 
kim, yi, western, japanese, school, chosŏn, tokyo, japan, oil, 15 0.32651322722328313
self, portraiture, portrait, na, ko, western, artist, autonomous, oil, academic